In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time

In [5]:
r1 = requests.get('https://www.spar.hu/sajtokapcsolat')
coverpage = r1.content
soup1 = BeautifulSoup(coverpage, 'html5lib')

coverpage_news = soup1.find_all('div', class_="accordion__inner-container")
tables = coverpage_news[1].find_all('tr')
links = []
dates = []

for i in range(3):
    tables = coverpage_news[i].find_all('tr')
    for m in range(len(tables)):
        dates.append(tables[m].find_all('td')[0].get_text())
        link = tables[m].find_all('td')[1].find('a').get('href','')
        if link.startswith('https://www.spar.hu'):
            links.append(link)
        else:
            links.append('https://www.spar.hu' + str(tables[m].find_all('td')[1].find('a').get('href','')))

for t in range(len(links)):
    print(dates[t], links[t])


2021.02.09. https://www.spar.hu/sajtokapcsolat/2021/hungaricool-by-sparversenyzokacapakkoezoettinnovativmagyartermek
2021.01.05. https://www.spar.hu/sajtokapcsolat/2021/megszulettek-a-2021-es-berek-a-sparnal
2020.12.22. https://www.spar.hu/sajtokapcsolat/2020/tizezer-csaladnaksegitazadnioroemakcio
2020.12.16. https://www.spar.hu/sajtokapcsolat/2020/adni-oroem-indulamagyarmaltaiszeretetszolgalatesasparjotekonysag
2020.12.03. https://www.spar.hu/sajtokapcsolat/2020/tovabb-bovuel-a-sparelektromosautotoeltohalozata
2020.11.26. https://www.spar.hu/sajtokapcsolat/2020/csaknem-3-milliardforintosberuhazasasparnalketujszupermarketetny
2020.11.25. https://www.spar.hu/sajtokapcsolat/2020/kezdodnek-az-uennepibevasarlasokszelesarukinalattalkeszueltfelas
2020.11.18. https://www.spar.hu/sajtokapcsolat/2020/feltoerekvo-magyarbeszallitokatkeresaspar
2020.11.11. https://www.spar.hu/sajtokapcsolat/2020/a-hazai-termekeketnepszerusitiaspar
2020.11.05. https://www.spar.hu/sajtokapcsolat/2020/vedjegyhasznala

In [9]:
cikkek_adatai = []
start = time.time()
counter = 0
for m in links:
    # link
    article = requests.get(m)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')

    #date
    date = dates[counter]
    
    # cím
    title_raw = soup_article.select("h1")
    if len(title_raw)>0:
        title = title_raw[0].get_text()
        title = title.strip()

    # törzsszöveg
    body = soup_article.find('div', class_="text__paragraph")
    if body is not None:
        x = body.select('p')
        list_paragraphs = []
        for p in np.arange(1, len(x)):
            paragraph = x[p].get_text()
            list_paragraphs.append(paragraph)
            final_article = " ".join(list_paragraphs)
    
    # putting it together
    cikkek_adatai.append([date, title, final_article, m])
    end = time.time()
    counter += 1
    print(str(counter) + '/' + str(len(links)), end - start)




1/112 0.3633608818054199
2/112 0.6852116584777832
3/112 1.0234653949737549
4/112 1.3641788959503174
5/112 1.79044771194458
6/112 2.2244274616241455
7/112 2.5613865852355957
8/112 2.878070831298828
9/112 3.2162728309631348
10/112 3.555466413497925
11/112 3.886045455932617
12/112 4.240065336227417
13/112 4.582056760787964
14/112 5.021053075790405
15/112 5.350181579589844
16/112 5.718207597732544
17/112 6.088108777999878
18/112 6.422989130020142
19/112 6.807066440582275
20/112 7.20889139175415
21/112 7.606039762496948
22/112 7.922076940536499
23/112 8.259984731674194
24/112 8.690366268157959
25/112 9.031055688858032
26/112 9.389430522918701
27/112 9.715163469314575
28/112 10.03087830543518
29/112 10.34101939201355
30/112 10.769430875778198
31/112 11.121992111206055
32/112 11.503966808319092
33/112 11.825157880783081
34/112 12.25396728515625
35/112 12.61739468574524
36/112 12.95840859413147
37/112 13.293002605438232
38/112 13.633021831512451
39/112 14.032978057861328
40/112 14.372057914733

In [8]:
len(cikkek_adatai)

27

In [10]:
df = pd.DataFrame(data=cikkek_adatai)
df.columns = ['dátum','cím', 'szöveg', 'link']
with pd.ExcelWriter('spar.xlsx') as writer:  
    df.to_excel(writer, sheet_name='hu')